In [2]:
import pandas as pd
import json as json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
import ast

In [4]:
# Cargar Data Sets
dfCreditsNormal = pd.read_csv('Data/dfCreditsNormal.csv', dtype=str, encoding='UTF-8', decimal='.', quotechar='"')
dfMovies = pd.read_csv('Data/dfMoviesNormal.csv', dtype=str, encoding='UTF-8', decimal='.', quotechar='"')

In [ ]:
dfCreditsNormal.drop(columns=['profile_path_cast', 'id_crew', 'cast_id_cast'], inplace=True)

In [ ]:
dfMoviesFinal.drop(columns=['id_collection', 'backdrop_path', 'id_genres', 'id_production_companies', 'iso_3166_1_production_countries', 'iso_639_1_spoken_languages' ], inplace=True)
dfMoviesFinal.drop(columns=['original_language', 'status'], inplace=True)

In [ ]:
dfMoviesSintetico = dfMoviesFinal.drop_duplicates(subset='id_pelicula', keep='first')

In [ ]:
dfMoviesSintetico

In [ ]:
# Verifica las primeras filas del DataFrame para asegurarte de que tiene las columnas correctas
print("DataFrame:")
print(dfMoviesSintetico.head())

# Verifica la descripción de las columnas seleccionadas
columns_to_describe = ['popularity', 'runtime', 'vote_average', 'release_year']
print("\nDescripción de las columnas seleccionadas:")
print(dfMoviesSintetico[columns_to_describe].describe())  # Selección correcta de columnas

# Crear el boxplot con matplotlib
plt.figure(figsize=(12, 8))
dfMoviesSintetico.boxplot(column=columns_to_describe)
plt.title('Boxplot de las columnas seleccionadas (matplotlib)')
plt.xlabel('Columnas')
plt.ylabel('Valores')
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario
plt.show()

# Convertir el DataFrame a formato largo para seaborn
df_long = dfMoviesSintetico.melt(value_vars=columns_to_describe)

# Crear el boxplot con seaborn
plt.figure(figsize=(12, 8))
sns.boxplot(x='variable', y='value', data=df_long)
plt.title('Boxplot de las columnas seleccionadas (seaborn)')
plt.xlabel('Columnas')
plt.ylabel('Valores')
plt.xticks(rotation=45)  # Rotar etiquetas si es necesario
plt.show()

In [ ]:
# Filtrar solo las columnas numéricas
numeric_columns = dfMoviesSintetico_scaled.select_dtypes(include='number').drop(columns=['id_pelicula', 'id_collection', 'return'], inplace=False)

# Agregar una columna categórica para usar en hue en el pairplot
# Aquí, puedes usar una columna categórica relevante. En este ejemplo, se usará 'name_genres', pero puedes cambiarlo según tu necesidad.
dfMovies_filtered = dfMoviesSintetico_scaled.copy()  # Crear una copia del DataFrame original
dfMovies_filtered = dfMovies_filtered[numeric_columns.columns.tolist() + ['name_genres']]

# Filtrar para eliminar filas con valores NaN en la columna categórica (si es necesario)
dfMovies_filtered = dfMovies_filtered.dropna(subset=['name_genres'])

# Crear el pairplot
sns.pairplot(dfMovies_filtered, hue='name_genres', palette='tab10')  # 'tab10' es una paleta con hasta 10 colores distintos

# Mostrar el gráfico
plt.show()
